In [1]:
import os 
import warnings
import numpy as np
import pandas as pd
import importlib as imp
from tqdm import tqdm, tqdm_notebook
warnings.simplefilter('ignore')
pd.options.display.max_columns = 100

**Задача**

*Ваша четвертая лабораторная работа будет заключаться в реализации одного из двух алгоритмов: <br>
метод имитации обжига или генетический алгоритм со встроенной эвристикой в виде локального поиска.  <br>
Алгоритмы реализуются для задачи Cell formation problem.* <br>

Реализовать можно на любом языке. <br>
Также нужно будет сравнить время работы алгоритма, а так же показать лучшее решение алгоритма, которое удалось найти вам. <br>

**Данные:**

Данные для работы прикрепил архивом к письму. **Структура такая:** <br>

m p (number of machines and parts) <br>
Next m rows: <br>
m(row number) list of parts processed by machine m separated by space <br>
e.g: <br>
1 9 17 19 31 33
means machine 1 processes parts 9 17 19 31 33 <br>

**Выходной формат для ответов:** <br>
Output file: instancename.sol (e.g. 20x20.sol) <br>
Output file format: <br>
m1_clusterId m2_clusterId ... - machines to clusters mapping <br>
p1_clusterId p2_clusterId ... - parts to clusters mapping <br>